In [1]:
import os
import tensorflow as tf
from tensorflow import contrib
import numpy as np
import pandas as pd
import scipy as sp 

from sklearn import cross_validation #, model_selection
#from sklearn.utils import shuffle
#import matplotlib as plt
#from matplotlib import pyplot
import matplotlib.pyplot as plt

logs_path = '/Users/dawnstear/desktop/tmp/tensorflow_logs/example_1/'
data  =  pd.read_csv('/Users/dawnstear/desktop/chop_cellpred/data.csv')

np.random.seed(42) # need to set multiple random seeds? # dont need to shuffle bc next_batch does already
#data_shuffled = shuffle(data)
data_shuffled = data

# Extract data and labels from gene expresssion matrix
X_ = data_shuffled.drop(['Labels','TYPE'],axis=1)
X_ = X_.iloc[:,:50]
y_ = data_shuffled['Labels']  # X_ and y_ are used b/c X and y are also standard name of placeholders in TF... 

cellcount, genecount = np.shape(X_)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_.values,y_.values,test_size=0.2)
                                                                # pass in X_.values so its a numpy.ndarray & not dframe
def next_batch(num, data, labels):  # from stack overflow, 
    # Return a total of `num` random samples and labels. 
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)    

/Users/dawnstear/desktop/tensorflow_update/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/dawnstear/desktop/tensorflow_update/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/dawnstear/desktop/tensorflow_update/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

# *-------Define architecture parameters--------*
num_input = genecount # *must preserve gene order when inputing data, take np.shape(X_) so no need for genecount -1 
n_hidden_1 = 64   
n_hidden_2 = 32
n_hidden_3 = 16
n_hidden_4 = 256
n_hidden_5 = 128
num_classes = 10 # == len(celldistro) # specify how many cell types ( # of labels in dataset)

# Create placeholders for train X,y and test X,y. Shape = (None, n_inputs)...
# bc we dont know how many samples we will have per batch yet
X = tf.placeholder(tf.float32, [None,num_input],name="X_data") # use -1 or None for dynamic batch size
y = tf.placeholder(tf.int64, [num_classes], name="y_labels")

# Build Architecture ''' Set weights to 0 option/reinit them '''
with tf.name_scope("dnn"):
    #with 
    #tf.reset_default_graph() #if condition, set to false
    layer1 = tf.layers.dense(inputs=X, units=n_hidden_1, activation=tf.nn.elu, name='Layer_1',reuse=tf.AUTO_REUSE) 
    dropout1 = tf.layers.dropout(inputs=layer1, rate=0.3,name='Dropout_1')
    layer2 = tf.layers.dense(inputs=dropout1,units=n_hidden_2, activation=tf.nn.elu,name='Layer_2',reuse=tf.AUTO_REUSE)#,reuse=True)
    dropout2 = tf.layers.dropout(inputs=layer2, rate=0.3,name='Dropout_2')
    layer3 = tf.layers.dense(inputs=dropout2, units=n_hidden_3, activation=tf.nn.elu,name='Layer_3',reuse=tf.AUTO_REUSE)#,reuse=True)
    output = tf.layers.dense(inputs=layer3,units=num_classes,activation=tf.nn.softmax,name='Output',reuse=tf.AUTO_REUSE)#,reuse=True)
    #output_tensor = tf.Print(output,[output]) # tf.Print it !!!!!!
    
######### Cost fn = softmax xentropy
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=output) # check sparse_xentropy docs
    loss = tf.reduce_mean(xentropy,name="loss_")          
######### Optimizer = SGD    
lr=0.0005
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(lr)
    training_op = optimizer.minimize(loss)  # pass in only variables you want to change !! *******
    
### Now how to evaluate...
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(output,y,1)  # check top_k docs
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32)) # compute roc metrics here?
# *------------End of Construction phase ----------*   

#*------------Begin Execution Stage----------------*
#saver = tf.train.Saver()

# *--------Set Some hyperparameters & initialize Metic Vectors --------*
n_epochs = 12
batch_size = 10
epochvec = range(1,n_epochs+1)
total_batch = (cellcount//batch_size)
display_epoch = 1
accTrain = []
accVal = []

In [7]:
init = tf.global_variables_initializer() # always do this right before you run sess or make sure all variables have been declared 
# clear out folder, or create new folder name with date/time/params eeach time, histogram of weights ????

tf.summary.scalar("Loss", loss)            # Create a summary to monitor cost (loss) tensor for tensorboard
tf.summary.scalar("Accuracy", accuracy)    # Create a summary to monitor accuracy tensor for tensorboard
merged_summary_op = tf.summary.merge_all() # Merge all summaries into a single op

# *--------------- Begin Run Session -------------------*
                              # ///config=tf.ConfigProto(log_device_placement=True)
with tf.Session() as sess:    # log_device checks if gpu or cpu is being used...print(sess.run(training_op,feed_dict={X:X_batch, y:y_batch}))
    tf.set_random_seed(1234)
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph()) # op to write logs to Tensorboard
    
    for epoch in range(n_epochs):
        #print('Starting Epoch %d ' % epoch)
        avg_loss = 0
        for iteration in range(cellcount//batch_size):
            
            X_batch, y_batch = next_batch(batch_size, X_train, y_train)
            _, c, summary = sess.run([training_op, loss, merged_summary_op],feed_dict={X:X_batch, y:y_batch}) # c for cost
            summary_writer.add_summary(summary, epoch * total_batch + iteration) # Write logs at every iteration
            avg_loss += c / total_batch # Compute average loss
                                                                    # Add these for Validation accuracy:
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch}) # acc_val = accuracy.eval(feed_dict={X:X_test, y:y_test})
        accTrain = np.append(accTrain,acc_train)                    # accVal = np.append(accVal,acc_val) 
                                                                    # print('Epoch:%d - Train Acc :%f - Validation Acc: %f' % (epoch, acc_train,acc_val))
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "Loss=", "{:.9f}".format(avg_loss))

    layer1_weights = sess.run('Layer_1/kernel:0') # save weight/biases for later inspection (must do sess.run() to get them)
    layer2_weights = sess.run('Layer_2/kernel:0') 
    layer3_weights = sess.run('Layer_3/kernel:0') # get output weights? 
    #layer1_bias = sess.run('Layer_1/bias:0')
         

# *--------------- Plot Results -------------------*   
fig, ax = plt.subplots()
ax.plot(epochvec,accTrain)
#ax.plot(epochvec,accVal)
ax.set(xlabel='Epoch', ylabel='Accuracies',title='Accuracy Over Training Phase') # include time//epoch//batch size in title
ax.grid()
#fig.savefig("/Users/dawnstear/desktop/tmp/")

# TENSORBOARD LINUX COMMAND (for macOS/safari, add the local host part): 
# tensorboard --logdir="/Users/dawnstear/desktop/tmp/tensorflow_logs/example_1/" --host localhost --port 8088
 

Epoch: 0001 Loss= 2.430806626
Epoch: 0002 Loss= 2.434556658
Epoch: 0003 Loss= 2.377173179
Epoch: 0004 Loss= 2.364881500
Epoch: 0005 Loss= 2.369656815
Epoch: 0006 Loss= 2.361259474
Epoch: 0007 Loss= 2.338480397
Epoch: 0008 Loss= 2.360881072
Epoch: 0009 Loss= 2.343766300
Epoch: 0010 Loss= 2.356025803
Epoch: 0011 Loss= 2.338512150
Epoch: 0012 Loss= 2.355979108


AttributeError: 'str' object has no attribute 'SerializeToString'

In [4]:
# check weights
print(tf.trainable_variables())
print("\n")
print(tf.report_uninitialized_variables())
print(np.shape(layer1_weights))
print(np.mean(layer1_weights)) # check reproducibility, use scatter or histo in tb 

[<tf.Variable 'Layer_1/kernel:0' shape=(50, 64) dtype=float32_ref>, <tf.Variable 'Layer_1/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'Layer_2/kernel:0' shape=(64, 32) dtype=float32_ref>, <tf.Variable 'Layer_2/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Layer_3/kernel:0' shape=(32, 16) dtype=float32_ref>, <tf.Variable 'Layer_3/bias:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'Output/kernel:0' shape=(16, 10) dtype=float32_ref>, <tf.Variable 'Output/bias:0' shape=(10,) dtype=float32_ref>]


Tensor("report_uninitialized_variables/boolean_mask/GatherV2:0", shape=(?,), dtype=string, device=/device:CPU:0)
(50, 64)
-0.00011928856


In [5]:
print(layer1_weights)
# To analyze weights from trial to trial, make histogram, #plt.hist(x,bins)
# Get max, min, median to create bins for histogram
print(type(layer1_weights))
max_1 = np.amax(layer1_weights) 
min_1 = np.amin(layer1_weights) 
mean_1 = np.median(layer1_weights)
print(max_1,min_1)
rank_1 = np.sort(layer1_weights) # no need  to sort for spearman rank coeff.
sp.stats.spearmanr()

# Every one of the 50 rows represents a gene and each column is a node, 1:64, if we want to silence a
# genes input then we need to set its whole row to zero

[[-0.18922128  0.18344417 -0.12067178 ...  0.02953382  0.11341352
   0.17954761]
 [-0.21845238  0.00102474 -0.01227096 ... -0.05355166 -0.06845389
   0.00173333]
 [-0.07991266 -0.11191159 -0.04416826 ...  0.04888756  0.11220614
  -0.04430334]
 ...
 [-0.21056946 -0.01913771 -0.14971414 ... -0.0214503  -0.21465547
  -0.19767147]
 [-0.0603504  -0.02652836  0.1475838  ...  0.12779285  0.17272927
   0.07958131]
 [-0.22618413 -0.11698591 -0.22707945 ... -0.21974349 -0.14670625
   0.09180107]]
<class 'numpy.ndarray'>
0.22929198 -0.2293377


In [6]:
#layer1.name
#sess.run('dnn_4/Layer_1/Elu:0')
#tf.boolean_mask
print(rank_1)

[[-0.22592145 -0.22432405 -0.21538211 ...  0.2137316   0.22062325
   0.22267553]
 [-0.2224403  -0.21845238 -0.21715821 ...  0.2232803   0.22488384
   0.22732513]
 [-0.22487088 -0.21608467 -0.21276365 ...  0.2196325   0.22100468
   0.22236815]
 ...
 [-0.22444175 -0.22347625 -0.21990612 ...  0.20545705  0.20990367
   0.22656004]
 [-0.21821412 -0.21415478 -0.20512629 ...  0.22503082  0.22690813
   0.22698866]
 [-0.22707945 -0.22618413 -0.21974349 ...  0.2006243   0.2224522
   0.22537404]]


In [ ]:
#rand shuffle seed needed 
#scatter plot for weights
#how does perturbation effect metabolism
# trey atticker paper
# mitochondria
# spearman correlation for correlation of ranking weights of a layer, just compare flattened np.arrays? closer to -1 or 1 means high pos or neg correlation

'''
  TODO:
      
 -check that labels were applied correctly    
 -next batch method, ok, use tf version?
 -use coo sparse dframe/matrix
 -softmax logits needs labels to be ints 0-9 so change labels to 0 through 9. DONE  
 -change shuffle method    
 -save trainedmodelwith date/time/params title 
 -implement Tensorboard DONE
 -Implement multithreading/queueing for cluster
 - implement pie chart of cell types and Venn diagram
 - plot accuracy as function of n epochs or n genes
 -kfold cross validation'''